In [1]:
import re
import json
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline 
from sklearn.impute import SimpleImputer 
from sklearn.compose import ColumnTransformer
from typing import Optional, Tuple, Dict, List
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split  
from sklearn.feature_selection import SelectFromModel 
from sklearn.preprocessing import OneHotEncoder, StandardScaler 
from sklearn.metrics import roc_auc_score, average_precision_score, classification_report, confusion_matrix

In [2]:
# ============================================
# CONFIG
# ============================================

# account-level table
DATA_PATH = r'N:\David Stewart\Desktop\88c6afda0b696ca552ccd7000b7ae067\Python\projectAtlas\C1B_OffUs_Attribute_Feature_Eng.xlsx' 
# data dictionary
DICT_PATH = r'N:\David Stewart\Desktop\88c6afda0b696ca552ccd7000b7ae067\Python\projectAtlas\ClarityDataDict.xlsx' 
# simple tag for outputs
OUTPUT_PREFIX = "run1"
# modeling target (1=good on C1B, 0=bad on C1B)
TARGET_COL = "perf_c1b" 
# columns to exclude from modeling
ID_COLS = ['base_ts','date_open','kob','experian_consumer_key', 'experian_trade_key', 'cl', 'present_status_enhanced', 'c1b_flag', 'vantage_v3_score'] 
# how many top features to print at the end
TOP_N_FEATURES = 20 

In [3]:
# ============================================
# LOAD DATA
# ============================================
def read_any(path: str) -> pd.DataFrame:
    if path.endswith(".parquet"):
        return pd.read_parquet(path)
    elif path.endswith(".csv"):
        return pd.read_csv(path)
    elif path.endswith(".xlsx"):
        return pd.read_excel(path)
    else:
        raise ValueError("DATA_PATH must be .csv or .parquet")

df_raw = read_any(DATA_PATH)
dict_df = read_any(DICT_PATH)

print("Loaded data:", df_raw.shape, "| dict rows:", dict_df.shape)

Loaded data: (33673, 1608) | dict rows: (1601, 10)


In [4]:
# ============================================
# NORMLIZE COLUMN NAMES 
# # ============================================
def normalize_colnames(cols):
    out = []
    for c in cols:
        c = str(c)
        # Take everytrhing after the last dot to drop schema
        if "." in c:
            c = c.split(".")[-1]
            # Lowercase and Trim
            c = c.strip().lower()
            out.append(c)
    return out
# Apply to dataframe and dictionary
df_raw.columns = normalize_colnames(df_raw.columns)
dict_df.columns = normalize_colnames(dict_df.columns)

In [5]:
# ============================================
# STEP 1 — BUILD TRADE-LEVEL perf_flag → CONSUMER-LEVEL FLAGS 
# # ============================================
# Map performance text → binary
perf_map = {
    'CURRENT': 1,
    '': None,
    '30 DPD': 0,
    '60 DPD': 0,
    '90 DPD': 0,
    '120+ DPD': 0,
    'DEROGATORY': 0
}
df = df_raw.copy()
df['perf_flag'] = df['present_status_enhanced'].map(perf_map)

# Consumer-level: Any Bad 
consumer_anybad = (
df.groupby("experian_consumer_key")["perf_flag"]
    .agg(lambda x: 0 if (0 in x.values) else (1 if 1 in x.values else None))
    .reset_index()
    .rename(columns={"perf_flag": "perf_anybad"})
)

# Consumer-level: C1B-centric target
c1b_only = df[df["c1b_flag"] == 1]
consumer_c1b = (
    c1b_only.groupby("experian_consumer_key")["perf_flag"]
    .agg(lambda x: 0 if (0 in x.values) else (1 if 1 in x.values else None))
    .reset_index()
    .rename(columns={"perf_flag": "perf_c1b"})
)

# Consumer-level: off-us performance
offus_only = df[df["c1b_flag"] == 0]
consumer_offus = (
    offus_only.groupby("experian_consumer_key")["perf_flag"]
    .agg([
        ("offus_anybad", lambda x: 1 if (0 in x.values) else 0),   # ever delinquent off-us
        ("offus_allgood", lambda x: 1 if (set(x.dropna()) == {1}) else 0)  # always current
    ])
    .reset_index()
)

# Deduplicate consumer attributes (they’re already consumer-level) 
attrs = df.drop_duplicates(subset=["experian_consumer_key"])

# 6. Merge everything into one table
consumer_level = (
    attrs
    .merge(consumer_anybad, on="experian_consumer_key", how="left")
    .merge(consumer_c1b, on="experian_consumer_key", how="left")
    .merge(consumer_offus, on="experian_consumer_key", how="left")
)

# Quick summary line for copy/paste
good = int((consumer_level["perf_c1b"] == 1).sum()) 
bad  = int((consumer_level["perf_c1b"] == 0).sum()) 
miss = int(consumer_level["perf_c1b"].isna().sum())
total = len(consumer_level)
print(f"\nSummary (perf_c1b): good={good} ({good/total:.1%}), bad={bad} ({bad/total:.1%}), missing={miss} ({miss/total:.1%}), total={total}")


Summary (perf_c1b): good=13641 (80.7%), bad=2041 (12.1%), missing=1219 (7.2%), total=16901


In [6]:
# Remove consumers without a C1B target (c1b_perf is NaN)
has_target_mask = consumer_level['perf_c1b'].notna()

# Do some checks
print(f"\nConsumers total: {len(consumer_level):,}")
print(f" - With C1B target: {has_target_mask.sum():,}")
print(f" - Missing C1B target (no C1B trade): {(~has_target_mask).sum():,}")

# Keep only rows that have a target for training
consumer_level = consumer_level.loc[has_target_mask].copy()

# Final re-check of balance after filtering
def pct(s): return (s / s.sum() * 100).round(2)
print("\n-- TARGET BALANCE (post-filter): perf_c1b --")
vc = consumer_level['perf_c1b'].value_counts(dropna=False)
print(vc)
print(pct(vc), "%")


Consumers total: 16,901
 - With C1B target: 15,682
 - Missing C1B target (no C1B trade): 1,219

-- TARGET BALANCE (post-filter): perf_c1b --
perf_c1b
1.0    13641
0.0     2041
Name: count, dtype: int64
perf_c1b
1.0    86.99
0.0    13.01
Name: count, dtype: float64 %


In [7]:
# ============================================
# STEP 2 — DICTIONARY-DRIVEN CLEANING (ONE DICTIONARY) 
# # ============================================
def parse_range(text: str) -> Tuple[Optional[float], Optional[float]]:
    if not isinstance(text, str):
        return None, None
    m = re.match(r"^\s*(-?\d+(?:\.\d+)?)\s*-\s*(-?\d+(?:\.\d+)?)\s*$", text.strip())
    if not m:
        return None, None
    lo, hi = float(m.group(1)), float(m.group(2))
    return (min(lo, hi), max(lo, hi))

def extract_codes(text: str) -> List[int]:
    if not isinstance(text, str):
        return []
    return [int(x) for x in re.findall(r"-?\d+", text)]

def no_trade_to_zero(text: str) -> bool:
    if not isinstance(text, str):
        return False
    t = text.lower()
    return ("no trade" in t) or ("no trade reported" in t)

def build_rules(dict_df: pd.DataFrame) -> Dict[str, dict]:
    rules = {}
    for _, row in dict_df.iterrows():
        col = str(row.get("field_name")).strip()
        if not col:
            continue
        lo, hi = parse_range(row.get("valid_values"))
        exc_text = row.get("default_exclusions")
        rules[col] = {
            "min": lo,
            "max": hi,
            "exc_codes": extract_codes(exc_text),
            "exc_to_zero": no_trade_to_zero(exc_text),
            "dtype_hint": str(row.get("dtype", "")).lower()
        }
    return rules

def clean_with_dictionary(
    df: pd.DataFrame,
    dict_df: pd.DataFrame,
    id_cols: Optional[List[str]] = None,
    target_col: Optional[str] = None,
    add_missing_flags: bool = True,
    clip_to_range: bool = True
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    rules = build_rules(dict_df)
    id_set = set(id_cols or [])
    cleaned = df.copy()
    report = []

    for col, rule in rules.items():
        if col not in cleaned.columns:
            continue
        if col in id_set or col == target_col:
            continue

        ser = cleaned[col]
        # numeric hint?
        is_num_hint = any(k in rule["dtype_hint"] for k in ["int","decimal","number","smallint","tinyint","bigint","float","double"])
        ser_num = pd.to_numeric(ser, errors="coerce") if (is_num_hint or ser.dtype != object) else pd.to_numeric(ser, errors="coerce")

        # If not numeric at all and not hinted numeric → keep as text (fill Unknown) and continue
        if ser_num.notna().sum() == 0 and not is_num_hint and ser.dtype == object:
            cleaned[col] = ser.fillna("Unknown")
            report.append({"column": col, "action": "categorical_fill_unknown"})
            continue

        # Apply exception codes
        exc_applied = 0; set_zero = 0; set_nan = 0
        if rule["exc_codes"]:
            mask = ser_num.isin(rule["exc_codes"])
            exc_applied = int(mask.sum())
            if exc_applied:
                if rule["exc_to_zero"]:
                    ser_num.loc[mask] = 0; set_zero += exc_applied
                else:
                    ser_num.loc[mask] = np.nan; set_nan += exc_applied

        # Missing flag BEFORE impute
        if add_missing_flags:
            cleaned[f"{col}_missing"] = ser_num.isna().astype(np.int8)

        # Impute: treat smallint ranges like counts → 0; otherwise median
        lo, hi = rule["min"], rule["max"]
        treat_as_count = (hi is not None and hi <= 10000 and "int" in rule["dtype_hint"])
        if treat_as_count:
            ser_num = ser_num.fillna(0)
        else:
            med = float(np.nanmedian(ser_num)) if ser_num.notna().any() else 0.0
            ser_num = ser_num.fillna(med)

        # Clip to stated range (if any)
        if clip_to_range and (lo is not None or hi is not None):
            ser_num = ser_num.clip(lower=lo, upper=hi)

        cleaned[col] = ser_num
        report.append({
            "column": col,
            "dtype_hint": rule["dtype_hint"],
            "exc_codes": "|".join(map(str, rule["exc_codes"])) if rule["exc_codes"] else "",
            "exc_to_zero": int(rule["exc_to_zero"]),
            "exc_applied": exc_applied,
            "set_zero": set_zero,
            "set_nan": set_nan,
            "range_min": lo, "range_max": hi
        })

    return cleaned, pd.DataFrame(report).sort_values("column")

# Apply dictionary to consumer_level
cleaned, clean_report = clean_with_dictionary(
    consumer_level,
    dict_df,
    id_cols=ID_COLS,
    target_col=TARGET_COL,
    add_missing_flags=True,
    clip_to_range=True
)

# Save intermediate artifacts (optional) 
cleaned.to_csv(f"{OUTPUT_PREFIX}.consumer_level.cleaned.csv", index=False) 
clean_report.to_csv(f"{OUTPUT_PREFIX}.clean_report.csv", index=False)

print("Cleaned shape:", cleaned.shape)

C:\Users\eappel\AppData\Local\Temp\ipykernel_18764\2691421812.py:84: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cleaned[f"{col}_missing"] = ser_num.isna().astype(np.int8)
C:\Users\eappel\AppData\Local\Temp\ipykernel_18764\2691421812.py:84: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  cleaned[f"{col}_missing"] = ser_num.isna().astype(np.int8)
C:\Users\eappel\AppData\Local\Temp\ipykernel_18764\2691421812.py:84: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

Cleaned shape: (15682, 3210)


In [8]:
# ============================================
# STEP 3 — MODEL: L1 feature selection → LightGBM (or RF) → Top N features 
# # ============================================
# Build X/y
assert TARGET_COL in cleaned.columns, f"Target '{TARGET_COL}' not found after cleaning"
y = cleaned[TARGET_COL].astype(int)
X = cleaned.drop(columns=[TARGET_COL] + [c for c in ID_COLS if c in cleaned.columns])

# Separate numeric vs categorical (objects) 
cat_cols = [c for c in X.columns if X[c].dtype == object] 
num_cols = [c for c in X.columns if c not in cat_cols]

# OneHotEncoder: handle new/old sklearn
try:
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True) 
except TypeError:
    # older sklearn
    ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)

num_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler(with_mean=True, with_std=True)),
])

cat_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("onehot", ohe),
])

pre = ColumnTransformer(
    transformers=[
        ("num", num_pipe, num_cols),
        ("cat", cat_pipe, cat_cols),
    ],
    remainder="drop"
)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# --- Fit the preprocessor first, then do L1 selection on the preprocessed matrix --- 
pre.fit(X_train, y_train) 
Xtr_pre = pre.transform(X_train) 
Xte_pre = pre.transform(X_test)

# L1 logistic for feature selection (wrapped inside SelectFromModel; no pipeline step for plain LogisticRegression) 
lasso = LogisticRegression(
    penalty="l1",
    solver="liblinear",
    max_iter=3000,
    class_weight="balanced"
)
selector = SelectFromModel(estimator=lasso, threshold="median") 
selector.fit(Xtr_pre, y_train)

Xtr_sel = selector.transform(Xtr_pre)
Xte_sel = selector.transform(Xte_pre)
print("Selected features (post-L1):", Xtr_sel.shape[1])

# Final model: LightGBM (fallback to RF if LGBM not installed)
try:
    from lightgbm import LGBMClassifier
    clf = LGBMClassifier(
        n_estimators=1000,
        learning_rate=0.03,
        num_leaves=64,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        class_weight="balanced",
        n_jobs=-1,
        random_state=42
    )
    model_name = "LightGBM"
except Exception:
    from sklearn.ensemble import RandomForestClassifier
    clf = RandomForestClassifier(
        n_estimators=500,
        min_samples_leaf=2,
        class_weight="balanced",
        n_jobs=-1,
        random_state=42
    )
    model_name = "RandomForest"

clf.fit(Xtr_sel, y_train)
proba = clf.predict_proba(Xte_sel)[:, 1]

roc = roc_auc_score(y_test, proba)
pr_auc = average_precision_score(y_test, proba) 
print(f"\n{model_name} ROC AUC: {roc:.4f} | PR AUC: {pr_auc:.4f}")

print("\n= Report @ 0.5 =")
pred = (proba >= 0.5).astype(int)
print(classification_report(y_test, pred, digits=3)) 
print("Confusion matrix @ 0.5:\n", confusion_matrix(y_test, pred))

# ------ Feature names and Top-N importances ------ 
# Build feature names from fitted preprocessor 
num_feature_names = num_cols[:]  # numeric pass-through names

cat_feature_names = []
if len(cat_cols):
    oh = pre.named_transformers_["cat"].named_steps["onehot"]
    cat_feature_names = list(oh.get_feature_names_out(cat_cols))

all_pre_names = num_feature_names + cat_feature_names

# Mask from selector
sel_mask = selector.get_support()
selected_feature_names = [n for n, keep in zip(all_pre_names, sel_mask) if keep]

# Importances
importances = getattr(clf, "feature_importances_", None) 
if importances is not None and len(importances) == len(selected_feature_names):
    order = np.argsort(importances)[::-1]
    print(f"\nTop {TOP_N_FEATURES} predictors:")
    for i in order[:TOP_N_FEATURES]:
        print(f"{selected_feature_names[i]:45s}  {importances[i]:.6f}")
else:
    print("\n[Note] Could not align importances with selected feature names. "
          "This can happen if the classifier lacks 'feature_importances_' or "
          "feature name lengths mismatch.")


C:\Users\eappel\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
C:\Users\eappel\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:1160: UserWarning: Inconsistent values: penalty=l1 with l1_ratio=0.0. penalty is deprecated. Please use l1_ratio only.
  warnings.warn(


Selected features (post-L1): 3209

RandomForest ROC AUC: 0.9804 | PR AUC: 0.9971

= Report @ 0.5 =
              precision    recall  f1-score   support

           0      0.801     0.730     0.764       612
           1      0.960     0.973     0.967      4093

    accuracy                          0.941      4705
   macro avg      0.881     0.852     0.865      4705
weighted avg      0.940     0.941     0.940      4705

Confusion matrix @ 0.5:
 [[ 447  165]
 [ 111 3982]]

Top 20 predictors:
perf_anybad                                    0.303536
perf_flag                                      0.150972
offus_allgood                                  0.093197
offus_anybad                                   0.091881
clall9549                                      0.004286
clinc9640                                      0.004245
clinc9810                                      0.004101
clntr9549                                      0.004020
clall9551                                      0.00392

In [ ]:
# 2) ========= DEFINE TARGET / DROP LEAKY COLS ========= 
TARGET = "good_perf"  # 1=good, 0=bad 
id_cols = ['a.base_ts','a.date_open','a.kob','a.experian_consumer_key', 'a.experian_trade_key', 'a.cl', 'a.present_status_enhanced', 'a.c1b_flag', 'a.vantage_v3_score']  

assert TARGET in df.columns, f"Target '{TARGET}' not found"

y = df[TARGET].astype(int)
X = df.drop(columns=[c for c in id_cols if c in df.columns] + [TARGET])


AssertionError: Target 'good_perf' not found

: 

In [ ]:
df['a.present_status_enhanced'].unique()